# Preprocess Data

In [1]:
import pandas as pd 
import numpy as np
from sklearn import preprocessing

## Data

In [2]:
# Information about attributes found at the link in crx.names file
attributeNames = ['A' + str(i+1) for i in range(16)]

df = pd.read_csv('../data/bronze/crx.data',
                   names = attributeNames,
                   header = None,
                   na_values = ['?'])

In [3]:
df.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,202.0,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,43.0,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,280.0,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,100.0,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,120.0,0,+


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   A1      678 non-null    object 
 1   A2      678 non-null    float64
 2   A3      690 non-null    float64
 3   A4      684 non-null    object 
 4   A5      684 non-null    object 
 5   A6      681 non-null    object 
 6   A7      681 non-null    object 
 7   A8      690 non-null    float64
 8   A9      690 non-null    object 
 9   A10     690 non-null    object 
 10  A11     690 non-null    int64  
 11  A12     690 non-null    object 
 12  A13     690 non-null    object 
 13  A14     677 non-null    float64
 14  A15     690 non-null    int64  
 15  A16     690 non-null    object 
dtypes: float64(4), int64(2), object(10)
memory usage: 86.4+ KB


In [5]:
df.describe(include='number').round(3)

,A2,A3,A8,A11,A14,A15
count,678.000,690.000,690.000,690.000,677.000,690.000
mean,31.568,4.759,2.223,2.400,184.015,1017.386
std,11.958,4.978,3.347,4.863,173.807,5210.103
min,13.750,0.000,0.000,0.000,0.000,0.000
25%,22.602,1.000,0.165,0.000,75.000,0.000
50%,28.460,2.750,1.000,0.000,160.000,5.000
75%,38.230,7.208,2.625,3.000,276.000,395.500
max,80.250,28.000,28.500,67.000,2000.000,100000.000


In [6]:
df.describe(include='object')

,A1,A4,A5,A6,A7,A9,A10,A12,A13,A16
count,678,684,684,681,681,690,690,690,690,690
unique,2,3,3,14,9,2,2,2,3,2
top,b,u,g,c,v,t,f,f,g,-
freq,468,519,519,137,399,361,395,374,625,383


### Categorical Data
How do the categorical features look?

In [7]:
for col in df.select_dtypes(include = 'object'):
    print('Category ' + col + ' : ')
    print(df[col].value_counts(),'\n')

Category A1 : 
A1
b    468
a    210
Name: count, dtype: int64 

Category A4 : 
A4
u    519
y    163
l      2
Name: count, dtype: int64 

Category A5 : 
A5
g     519
p     163
gg      2
Name: count, dtype: int64 

Category A6 : 
A6
c     137
q      78
w      64
i      59
aa     54
ff     53
k      51
cc     41
m      38
x      38
d      30
e      25
j      10
r       3
Name: count, dtype: int64 

Category A7 : 
A7
v     399
h     138
bb     59
ff     57
j       8
z       8
dd      6
n       4
o       2
Name: count, dtype: int64 

Category A9 : 
A9
t    361
f    329
Name: count, dtype: int64 

Category A10 : 
A10
f    395
t    295
Name: count, dtype: int64 

Category A12 : 
A12
f    374
t    316
Name: count, dtype: int64 

Category A13 : 
A13
g    625
s     57
p      8
Name: count, dtype: int64 

Category A16 : 
A16
-    383
+    307
Name: count, dtype: int64 



### Redundant Features
The dataset does not contain:
- IDs
- Constants
- Time series information

However, columns A4 and A5 seem to have the same amount of variables, with the exact same value count distribution. This indicates there might be a repeated column.

In [8]:
A4 = df['A4'].copy()
A5 = df['A5'].copy()

A5 = A5.replace({'g':'u', 'p':'y', 'gg':'l'})

print(A4.head(10).values)
print(A5.head(10).values)

print(A4.equals(A5))

['u' 'u' 'u' 'u' 'u' 'u' 'u' 'u' 'y' 'y']
['u' 'u' 'u' 'u' 'u' 'u' 'u' 'u' 'y' 'y']
True


columns A4 and A5 are the same, therefore, we can drop either column as to prevent redundancy.

In [9]:
df = df.drop(columns = 'A5')

### Handling Null Values

In [10]:
df_imputed = df.copy()

for col in df_imputed.select_dtypes(include = 'object'):
    mode = df_imputed[col].mode()[0]
    print('Category ' + col + ' mode: ' + mode)
    df_imputed[col] =  df_imputed[col].fillna(mode)

Category A1 mode: b
Category A4 mode: u
Category A6 mode: c
Category A7 mode: v
Category A9 mode: t
Category A10 mode: f
Category A12 mode: f
Category A13 mode: g
Category A16 mode: -


In [11]:
for col in df_imputed.select_dtypes(include = 'number'):
    median = np.round(df_imputed[col].median(),3)
    print('Category ' + col + ' median: ' + str(median))
    df_imputed[col] =  df_imputed[col].fillna(median)

Category A2 median: 28.46
Category A3 median: 2.75
Category A8 median: 1.0
Category A11 median: 0.0
Category A14 median: 160.0
Category A15 median: 5.0


In [12]:
# Checking n/a value counts
df_imputed.isna().sum()

A1     0
A2     0
A3     0
A4     0
A6     0
A7     0
A8     0
A9     0
A10    0
A11    0
A12    0
A13    0
A14    0
A15    0
A16    0
dtype: int64

All null values have been handled from the dataset.

## Feature Encoding

### Integer Encoding

Let's discretise column A2 by equal-frequency binning with "low", "medium", and "high" and then apply integer encoding.

We can also go ahead and perform a manual replace encoding for the A16 target feature, where we assume that '+' indicates a credit approval (1), and '-' indicates no approval (0).

In [13]:
df_encoded = df_imputed.copy()
df_encoded['A2'] = pd.qcut(df_encoded['A2'], q = 3, labels =['low','medium','high'])

In [14]:
df_encoded['A2'].value_counts()

A2
medium    231
low       230
high      229
Name: count, dtype: int64

In [15]:
level_mapping = {'low':0, 'medium':1, 'high':2 }
df_encoded['A2'] = df_encoded['A2'].replace(level_mapping)
df_encoded['A16'] = df_encoded['A16'].replace({'+':1, '-':0})

In [16]:
df_encoded.head()

,A1,A2,A3,A4,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,b,1,0.000,u,w,v,1.25,t,t,1,f,g,202.0,0,1
1,a,2,4.460,u,q,h,3.04,t,t,6,f,g,43.0,560,1
2,a,1,0.500,u,q,h,1.50,t,f,0,f,g,280.0,824,1
3,b,1,1.540,u,w,v,3.75,t,t,5,t,g,100.0,3,1
4,b,0,5.625,u,w,v,1.71,t,f,0,f,s,120.0,0,1


Column A2 has been equal frequency binned with 0 representing low values, 1 representing medium values and 2 representing high values.

### One Hot Encodin

Now that Category A2 has been integer encoded, we can go ahead and perform one hot encoding on the rest of the categorical features. We should separate the target feature (A16) from the rest of the dataset.

In [17]:
# Removal of the target feature, to be merged later
target = df_encoded['A16']
df_ohe = df_encoded.drop(columns = 'A16')

In [18]:
for col in df_ohe.select_dtypes(include ='object'):
    if(len(df_ohe[col].unique()) ==  2):
        df_ohe[col] = pd.get_dummies(df_ohe[col], drop_first = True)

df_ohe = pd.get_dummies(df_ohe)     

print(df_ohe.shape)

(690, 41)


In [19]:
df_ohe.head()

,A1,A3,A8,A9,A10,A11,A12,A14,A15,A2_0,...,A7_ff,A7_h,A7_j,A7_n,A7_o,A7_v,A7_z,A13_g,A13_p,A13_s
0,True,0.000,1.25,True,True,1,False,202.0,0,False,...,False,False,False,False,False,True,False,True,False,False
1,False,4.460,3.04,True,True,6,False,43.0,560,False,...,False,True,False,False,False,False,False,True,False,False
2,False,0.500,1.50,True,False,0,False,280.0,824,False,...,False,True,False,False,False,False,False,True,False,False
3,True,1.540,3.75,True,True,5,True,100.0,3,False,...,False,False,False,False,False,True,False,True,False,False
4,True,5.625,1.71,True,False,0,False,120.0,0,True,...,False,False,False,False,False,True,False,False,False,True


## Scaling

### Standard Scaling

Let's use Standard Scaling in order for all the variables to be distributed on the same scale for use in models.

In [20]:
df_scaled = df_ohe.copy()

# Saving the column names prior to scaling
col_names = df_scaled.columns

# Perform standardised scaling
df_standard_scaled = preprocessing.StandardScaler().fit_transform(df_scaled)

# Convert back to a Data Frame, with correct column names
df_scaled = pd.DataFrame(df_standard_scaled,
                    columns = col_names)

print(df_scaled.shape)

(690, 41)


In [21]:
df_scaled.sample(n=15, random_state = 1)

,A1,A3,A8,A9,A10,A11,A12,A14,A15,A2_0,...,A7_ff,A7_h,A7_j,A7_n,A7_o,A7_v,A7_z,A13_g,A13_p,A13_s
666,-1.511858,1.405407,-0.590118,-1.047504,-0.864196,-0.493887,1.087908,-0.020703,-0.195413,1.414214,...,-0.300079,-0.5,-0.108306,-0.07636,-0.053916,0.831370,-0.108306,0.322490,-0.108306,-0.300079
520,0.661438,0.551059,-0.216324,0.954650,1.157144,-0.288101,-0.919195,-0.136938,-0.150468,1.414214,...,-0.300079,-0.5,-0.108306,-0.07636,-0.053916,0.831370,-0.108306,0.322490,-0.108306,-0.300079
681,0.661438,-0.755590,0.232229,-1.047504,-0.864196,-0.493887,-0.919195,-0.043950,-0.092270,-0.707107,...,-0.300079,2.0,-0.108306,-0.07636,-0.053916,-1.202834,-0.108306,0.322490,-0.108306,-0.300079
23,-1.511858,1.958221,0.257647,0.954650,1.157144,-0.288101,-0.919195,-0.369408,-0.193301,-0.707107,...,-0.300079,2.0,-0.108306,-0.07636,-0.053916,-1.202834,-0.108306,0.322490,-0.108306,-0.300079
65,0.661438,-0.647038,-0.204362,0.954650,1.157144,-0.288101,1.087908,1.955289,9.408288,-0.707107,...,-0.300079,-0.5,-0.108306,-0.07636,-0.053916,0.831370,-0.108306,0.322490,-0.108306,-0.300079
339,0.661438,-0.353544,-0.440600,-1.047504,-0.864196,-0.493887,1.087908,0.676706,-0.182544,-0.707107,...,-0.300079,-0.5,-0.108306,-0.07636,-0.053916,0.831370,-0.108306,0.322490,-0.108306,-0.300079
56,0.661438,-0.956613,-0.639459,0.954650,-0.864196,-0.493887,-0.919195,-1.066817,-0.195413,1.414214,...,-0.300079,-0.5,-0.108306,-0.07636,-0.053916,0.831370,-0.108306,0.322490,-0.108306,-0.300079
394,0.661438,-0.705334,-0.590118,-1.047504,-0.864196,-0.493887,-0.919195,-1.066817,-0.157959,-0.707107,...,-0.300079,-0.5,-0.108306,-0.07636,-0.053916,0.831370,-0.108306,0.322490,-0.108306,-0.300079
47,0.661438,-0.060051,1.141296,0.954650,1.157144,0.123472,-0.919195,0.740635,-0.137791,-0.707107,...,-0.300079,2.0,-0.108306,-0.07636,-0.053916,-1.202834,-0.108306,0.322490,-0.108306,-0.300079
358,0.661438,-0.353544,-0.615536,-1.047504,-0.864196,-0.493887,1.087908,-0.369408,-0.195413,-0.707107,...,-0.300079,-0.5,-0.108306,-0.07636,-0.053916,0.831370,-0.108306,0.322490,-0.108306,-0.300079


## Target Feature

Let's add the target feature back to the standardised dataset. The final dataset has been fully cleaned.

In [22]:
df_clean = df_scaled.assign(target = target.values)

df_clean = df_clean.round(5)

df_clean.sample(n = 15, random_state = 1)

,A1,A3,A8,A9,A10,A11,A12,A14,A15,A2_0,...,A7_h,A7_j,A7_n,A7_o,A7_v,A7_z,A13_g,A13_p,A13_s,target
666,-1.51186,1.40541,-0.59012,-1.04750,-0.86420,-0.49389,1.08791,-0.02070,-0.19541,1.41421,...,-0.5,-0.10831,-0.07636,-0.05392,0.83137,-0.10831,0.32249,-0.10831,-0.30008,0
520,0.66144,0.55106,-0.21632,0.95465,1.15714,-0.28810,-0.91920,-0.13694,-0.15047,1.41421,...,-0.5,-0.10831,-0.07636,-0.05392,0.83137,-0.10831,0.32249,-0.10831,-0.30008,1
681,0.66144,-0.75559,0.23223,-1.04750,-0.86420,-0.49389,-0.91920,-0.04395,-0.09227,-0.70711,...,2.0,-0.10831,-0.07636,-0.05392,-1.20283,-0.10831,0.32249,-0.10831,-0.30008,0
23,-1.51186,1.95822,0.25765,0.95465,1.15714,-0.28810,-0.91920,-0.36941,-0.19330,-0.70711,...,2.0,-0.10831,-0.07636,-0.05392,-1.20283,-0.10831,0.32249,-0.10831,-0.30008,1
65,0.66144,-0.64704,-0.20436,0.95465,1.15714,-0.28810,1.08791,1.95529,9.40829,-0.70711,...,-0.5,-0.10831,-0.07636,-0.05392,0.83137,-0.10831,0.32249,-0.10831,-0.30008,1
339,0.66144,-0.35354,-0.44060,-1.04750,-0.86420,-0.49389,1.08791,0.67671,-0.18254,-0.70711,...,-0.5,-0.10831,-0.07636,-0.05392,0.83137,-0.10831,0.32249,-0.10831,-0.30008,0
56,0.66144,-0.95661,-0.63946,0.95465,-0.86420,-0.49389,-0.91920,-1.06682,-0.19541,1.41421,...,-0.5,-0.10831,-0.07636,-0.05392,0.83137,-0.10831,0.32249,-0.10831,-0.30008,1
394,0.66144,-0.70533,-0.59012,-1.04750,-0.86420,-0.49389,-0.91920,-1.06682,-0.15796,-0.70711,...,-0.5,-0.10831,-0.07636,-0.05392,0.83137,-0.10831,0.32249,-0.10831,-0.30008,0
47,0.66144,-0.06005,1.14130,0.95465,1.15714,0.12347,-0.91920,0.74063,-0.13779,-0.70711,...,2.0,-0.10831,-0.07636,-0.05392,-1.20283,-0.10831,0.32249,-0.10831,-0.30008,1
358,0.66144,-0.35354,-0.61554,-1.04750,-0.86420,-0.49389,1.08791,-0.36941,-0.19541,-0.70711,...,-0.5,-0.10831,-0.07636,-0.05392,0.83137,-0.10831,0.32249,-0.10831,-0.30008,0


__The dataset is now clean.__ It has no missing values, all columns are numerical and have been scaled, the target feature is correctly label encoded. It can now be used in a machine learning model without any further data preprocessing.

## Export

In [23]:
df_clean.to_csv('../data/silver/crx.csv', index = False)
df_clean.to_parquet('../data/silver/crx.parquet', index=False)